# DB Cleaning

Input: raw DB from scraper  
Goal:  
- clean all fields  
- extract features  
- write back to DB

In [1]:
import sqlite3
import pandas as pd

In [2]:
def create_connection(db_file):
    """ create a database connection to the SQLite database
        specified by the db_file
    :param db_file: database file
    :return: Connection object or None
    """
    conn = None
    try:
        conn = sqlite3.connect(db_file)
    except Error as e:
        print(e)
 
    return conn

# Clean all fields 

In [3]:
# INPUTS
table_name = 'ARTICLES_538'
staging_table = 'S_ARTICLES_538'

In [4]:
conn = create_connection('../../scraping538.db')
# data = pd.read_sql_query("SELECT * FROM TEST_TBL", conn)
query = "SELECT * FROM " + staging_table + " WHERE PROCESSED = 0"
data = pd.read_sql(query, con=conn)
data.drop("PROCESSED", axis=1, inplace=True)
# data.shape



# date
data['date'] = pd.to_datetime(data['date'])
data.rename(columns={"date": "DATE"}, inplace=True)

# date_hour
# (you can have NaT)
data['date_hour'] = data['date_hour'].str.replace('.','')
data['date_hour'] = pd.to_datetime(data['date_hour'], 
                                   format='%b %d, %Y-%I:%M %p', 
                                   errors='coerce')
data.rename(columns={"date_hour": "DATE_HOUR"}, inplace=True)

# date_import
data['date_import'] = pd.to_datetime(data['date_import'])
data.rename(columns={"date_import": "DATE_IMPORT"}, inplace=True)

# define PK
data['PK_ID'] = data['DATE'].dt.strftime('%d-%m-%Y_') + data['title']

data['TEXTRAZOR_JSON_SAVED'] = 0

data.head(2)

,title,DATE,hour,DATE_HOUR,author,filed_under,article_text,article_text_without_children,mini_bio,url,project,spider,server,DATE_IMPORT,PK_ID,new
0,Why Even More Democrats Are Thinking About Ent...,2019-11-12,3:22 PM,2019-11-12 15:22:00,Perry Bacon Jr.,2020 Election,There have been a flurry of recent stories in ...,There have been a flurry of recent stories in ...,Perry Bacon Jr. is a senior writer for FiveThi...,https://fivethirtyeight.com/features/why-are-d...,scrapping_538,all_articles_from_one_page,mrmoe,2019-11-12 22:13:35.156587,12-11-2019_Why Even More Democrats Are Thinkin...,0
1,Want To Confuse An NBA Defense? Have A Guard S...,2019-11-12,2:58 PM,2019-11-12 14:58:00,Jared Dubin,NBA,Picture a pick and roll in your mind’s eye. Wh...,Picture a pick and roll in your mind’s eye. Wh...,Jared Dubin is a New York writer and lawyer. H...,https://fivethirtyeight.com/features/want-to-c...,scrapping_538,all_articles_from_one_page,mrmoe,2019-11-12 22:13:35.179565,12-11-2019_Want To Confuse An NBA Defense? Hav...,0


# Write back to DB - 

**Iteration** through each record and write to DB

In [5]:
num_rejected = 0

for index in range(data.shape[0]):
    
    one_rec = data.loc[[index]]
    
    try:
        one_rec.to_sql(table_name, con=conn, if_exists='append', index=False)
        pass
    except sqlite3.IntegrityError as err:
        print('*** IntegrityError  --  ', one_rec['PK_ID'][index])
        num_rejected += 1
    finally:
        conn.commit()

*** IntegrityError  --   08-11-2019_How Seriously Should We Take Michael Bloomberg’s Potential 2020 Run?
*** IntegrityError  --   11-11-2019_How To Improve The Conversation About Electability
*** IntegrityError  --   11-11-2019_Running For President May Make You More Unpopular In Your Home State
*** IntegrityError  --   11-11-2019_Jeff Sessions Might Struggle To Win Back His Old Senate Seat
*** IntegrityError  --   11-11-2019_LeBron Is Great At Everything — Even Point Guard
*** IntegrityError  --   08-11-2019_Championships Aren’t Won On Paper. But What If They Were?
*** IntegrityError  --   11-11-2019_The Good, The Bad And The Weird Of NFL Week 10
*** IntegrityError  --   07-11-2019_If Alabama Loses To LSU, Will It Actually Miss The Playoff?
*** IntegrityError  --   07-11-2019_Who’s Up And Who’s Down In Our NBA Player Ratings
*** IntegrityError  --   08-11-2019_Post-Debate Polls Show Sanders Solidly In Third
*** IntegrityError  --   07-11-2019_The 7 Ways Impeachment Could Shape The 202

In [6]:
conn.execute('UPDATE ' + staging_table + ' SET PROCESSED = 1 WHERE PROCESSED = 0;')
conn.commit()
conn.close()

### Summary

In [7]:
print('# of articles loaded:       ', data.shape[0])
print('# of articles "integrated": ', data.shape[0] - num_rejected)
print('# of articles rejected:     ', num_rejected)

# of articles loaded:        51
# of articles "integrated":  0
# of articles rejected:      51


In [8]:
import sys
sys.exit()

SystemExit: 

D:\Programs\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2889: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


# Write back to DB - à la M3O

Does not really work  
Does only work for all records

In [ ]:
# INPUTS
table_name = 'ARTICLES_538'

In [ ]:
column_part = ', '.join(data.columns)
value_part = ', '.join(['"a"']*14)

In [ ]:
col_brackets = ['[' + items + ']' for items in data.columns]
col_brackets = ','.join(col_brackets)
col_brackets

In [ ]:
placeholders = ['?' for items in data.columns]
placeholders = ','.join(placeholders)
placeholders

In [ ]:
string_insert = "INSERT INTO " + table_name + " (" + col_brackets + ") VALUES ("
string_insert

In [ ]:
for index, row in data.iterrows():
    rows = [row[items].__str__() for items in data.columns]
    rows = str(rows).replace("''", 'null')
    rows = rows[1:-1]
    
    string_insert_all = string_insert + rows + ');'
    
    try:
        conn.execute(string_insert_all)
        pass
    except sqlite3.IntegrityError as err:
        print('*** IntegrityError  --  ', row['PK_ID'])
    finally:
        conn.commit()

In [ ]:
conn.execute(string_insert_all)

In [ ]:
string_insert_all